In [4]:
import tensorflow as tf
import datetime
import abc
from tqdm import tqdm

In [5]:
load_ext tensorboard

In [6]:
import pickle
import tensorflow as tf
# Wczytanie zbioru danych
dataset_path = 'dataset.pkl'

with open(dataset_path, 'rb') as f:
    dataset = pickle.load(f)  

images_raw_train = dataset['train']['X']
images_raw_test = dataset['test']['X']
images_raw_valid = dataset['validation']['X']

labels = dataset['train']['y']
labels_test = dataset['test']['y']
lebels_valid = dataset['validation']['y']
# Ustawienie ziarna
tf.random.set_seed(1234)

# Zmiana rozmiaru obrazu z 224x224 do 64x64
images = tf.image.resize(
    images_raw_train, [64, 64], 
    method=tf.image.ResizeMethod.NEAREST_NEIGHBOR, 
    preserve_aspect_ratio=False,
    name=None
)

images_test = tf.image.resize(
    images_raw_test, [64, 64], 
    method=tf.image.ResizeMethod.NEAREST_NEIGHBOR, 
    preserve_aspect_ratio=False,
    name=None
)

images_valid = tf.image.resize(
    images_raw_valid, [64, 64], 
    method=tf.image.ResizeMethod.NEAREST_NEIGHBOR, 
    preserve_aspect_ratio=False,
    name=None
)

# Zmiana typu obrazów z int na float i normalizacja wartości pikseli z przedziału 0-255 do przedziału 0-1
images = tf.cast(images, tf.float32) / 255.0
images_test = tf.cast(images_test, tf.float32) / 255.0
images_valid = tf.cast(images_valid, tf.float32) / 255.0
# Zmiana wymiarowości z [8870] na [8870, 1] w celu zastosowania `SparseCategoricalCrossentropy`
labels = tf.reshape(labels, [-1, 1])
labels_test = tf.reshape(labels_test, [-1, 1])
lebels_valid = tf.reshape(lebels_valid, [-1, 1])
# Do iterowania zbioru danych i podzielenia na paczki (ang. batch) można wykorzystać interfejs zdefiniowany w `tensorflow.data.Dataset`

train_ds = tf.data.Dataset.from_tensor_slices((images, labels)) 
train_ds = train_ds.shuffle(buffer_size=len(images)).batch(batch_size=100)

In [7]:
import abc



class AbstractLayer(tf.keras.layers.Layer):
    """Abstract Layer."""
    
    def __init__(self):
        """Inits the class."""
        super(AbstractLayer, self).__init__()
    
    @abc.abstractmethod
    def call(self, inputs):
        """Makes forward pass of the layer."""
        pass


class AbstractModel(tf.keras.Model):
    """Abstract model."""

    def __init__(self):
        """Inits the class."""
        super(AbstractModel, self).__init__()

    @abc.abstractmethod
    def call(self, inputs, training=False):
        """Makes forward pass of the network."""
        pass

    @abc.abstractmethod
    def fit(self, **kwargs):
        """Fits the model.

        Replaces original fit method.
        
        More information can be found in documentation:
        https://www.tensorflow.org/guide/eager
        https://www.tensorflow.org/guide/autodiff
        """
        pass

    @abc.abstractmethod
    def predict(self, x):
        """Predicts x"""
        pass

In [8]:
import abc
import tensorflow as tf

class MLP_Layer(AbstractLayer):
    
    def __init__(self, units):
        """Inits the class."""
        self.units = units
        super(MLP_Layer, self).__init__()
    
    def build(self, input_shape):
        """Here we build our layer. Here will be create Weights and baiases"""
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                      initializer=tf.random_uniform_initializer,
                                      trainable=True)
        
        self.b = self.add_weight(shape=(self.units,),initializer='random_normal',trainable=True)
        
    def call(self, inputs):
        """Makes forward pass of the layer."""
        #y = a*x +b  -> weights * input + bias
        ax = tf.matmul(inputs, self.w)
        y = ax + self.b
        return y    

In [9]:
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
class MLP(tf.keras.Model):
    
    def __init__(self):
        super(MLP, self).__init__()
        self.optim = tf.keras.optimizers.Adam(learning_rate=1e-3)
        self.loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True) #for one hot encoding
        self.full_connected_1 = MLP_Layer(units=256)
        #full connected 14 units Softmax
        self.full_connected_2 = MLP_Layer(units=14)

    def call(self, inputs, training=False):
        """Makes forward pass of the network."""
        x = tf.reshape(inputs,[inputs.shape[0],64*64*3])
        x = self.full_connected_1(x)
        x = tf.nn.relu(x) #Activation 1 layer
        x = self.full_connected_2(x)
        x = tf.nn.softmax(x) #Activation 2 layer
        x=tf.keras.backend.sum(loss)
        return x

    def fit(self, **kwargs):
        history = {'loss':[],
                   'accuracy':[],
                   'Fscore':[]}
        batch_size = 100
        #print(type(history['loss']))
        train_dataset= kwargs['dataset']
        test_dataset= kwargs['test_ds']
        valid_dataset = kwargs['valid_ds']
        num_epochs = kwargs['epochs']
        loss_list = []
        NUMBER_OF_CLASSES = 14
        prec = tf.keras.metrics.Precision()
        recall = tf.keras.metrics.Recall()
        acc = tf.keras.metrics.Accuracy()
        if(train_dataset!=None):
            for epoch in tqdm(range(num_epochs)):
                acc.reset_states() #reset acc
                prec.reset_states()
                recall.reset_states()
                
                train_ds = tf.data.Dataset.from_tensor_slices((images, labels)) 
                size=len(train_dataset)
                train_ds = train_ds.shuffle(buffer_size=size).batch(batch_size=batch_size)
                
                loss_sum = 0
                for x, y in train_ds:
                    with tf.GradientTape() as tape:
                        y_onehot = tf.reshape(tf.one_hot(y,depth=NUMBER_OF_CLASSES),(len(y),NUMBER_OF_CLASSES))
                        y_prediction = self.call(x,training=True)
                        
                        acc.update_state(y_pred=y_prediction,y_true=y_onehot)
                        recall.update_state(y_pred=y_prediction,y_true=y_onehot)
                        prec.update_state(y_pred=y_prediction,y_true=y_onehot)
                        loss = self.loss_fn(y_onehot,y_prediction)
                        loss_sum+=tf.keras.backend.sum(loss)
                        
                    grads = tape.gradient(loss, self.trainable_variables)
                    self.optim.apply_gradients(zip(grads,self.trainable_variables))
                r = recall.result().numpy()
                p = prec.result().numpy()
                F1 = 2*p*r
                F1 = F1/(p+r)
                history['accuracy'].append(m.result().numpy())
                history['loss'].append((loss_sum/size).numpy())
                history['Fscore'].append(F1)
        else:
            print('provide dataset')
              
        return history
        pass

    def predict(self, x):
        """Predicts x"""
        pred = self(x,training=False)
        return pred

In [10]:
model=MLP()
history = model.fit(dataset=images,epochs=25,test=images_test_ds,valid=images_valid)

NameError: name 'images_test_ds' is not defined

In [1]:
import tensorflow as tf
Cacc = tf.keras.metrics.SparseCategoricalAccuracy()
acc = tf.keras.metrics.Accuracy()

In [134]:
y_true = tf.constant([[0,0,0,0,1]])

In [135]:
y_pred = tf.constant([[1,0,0,0,1]])

In [136]:
Cacc.update_state(y_true=y_true,y_pred=y_pred)
acc.update_state(y_true=y_true,y_pred=y_pred)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=5.0>

In [137]:
print(Cacc.result().numpy())
print(acc.result().numpy())

0.0
0.8


In [138]:
Cacc.reset_states()
acc.reset_states()

In [139]:
y_true = tf.constant([[0,0,0,0,1]])
y_pred = tf.constant([[1,0,0,0,1]])
recall = tf.keras.metrics.Recall()
recall.update_state(y_true=y_true,y_pred=y_pred)
res = recall.result()
recall.reset_states()
print(res)

tf.Tensor(1.0, shape=(), dtype=float32)
